In [ ]:
import torch
from torch_geometric.datasets import MovieLens1M, MovieLens100K


# Load MovieLens 1M dataset (adjust root if needed) push test
#dataset = MovieLens1M(root='/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg')
dataset= MovieLens100K(root="/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg")
print(dataset[0])


/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from ultra import datasets

dataset = datasets.Yelp18(root = "/itet-stor/trachsele/net_scratch/tl4rec/model_outputs/data")
print (f"dataset[0].edge_index.shape{: dataset[0].edge_index.shape}")

dataset[0].edge_index.shapetorch.Size([2, 2194014])


In [1]:
import torch

# Parameters
batch_size = 2
num_negatives = 2
num_users = 3
num_items = 3
dim = 2

# Embeddings for users and items (3 users, 3 items, embedding dimension 2)
user_embedding = torch.tensor([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]])
item_embedding = torch.tensor([[0.7, 0.8], [0.9, 1.0], [1.1, 1.2]])

# h_index and t_index based on the edges and negative samples
# Each row represents a batch entry, and each column is a negative sample
# Here, `0` and `1` are valid head indices (users), and we corrupt them by keeping valid and invalid tails
h_index = torch.tensor([
    [0, 0, 0],  # For first edge (0, 0, 3) with two negatives
    [1, 2, 0]   # For second edge (1, 0, 4) with two negatives
])
t_index = torch.tensor([
    [3, 4, 5],  # Original (0, 0, 3), and corrupted tails [4, 2]
    [4, 4, 4]   # Original (1, 0, 4), and corrupted tails [5, 0]
])

# Gather head node embeddings
# (num_nodes, dim)
index_temp = h_index.unsqueeze(-1).expand(-1, -1, user_embedding.shape[-1])
h_embeddings = user_embedding.unsqueeze(0).expand(batch_size,-1,-1).gather(1, h_index.unsqueeze(-1).expand(-1, -1, dim))

# Adjust `t_index` to map to item IDs by subtracting `num_users`
index_temp = (t_index - num_users).clamp(min=0)
t_embeddings = item_embedding.unsqueeze(0).expand(batch_size,-1,-1).gather(1, index_temp.unsqueeze(-1).expand(-1, -1, dim))

print("User Embedding Tensor:\n", user_embedding)
print("Item Embedding Tensor:\n", item_embedding)
print("Head Embeddings:\n", h_embeddings)
print("Tail Embeddings:\n", t_embeddings)
print("size head Embeddings:\n", h_embeddings.shape)
print("size Tail Embeddings:\n", t_embeddings.shape)


User Embedding Tensor:
 tensor([[0.1000, 0.2000],
        [0.3000, 0.4000],
        [0.5000, 0.6000]])
Item Embedding Tensor:
 tensor([[0.7000, 0.8000],
        [0.9000, 1.0000],
        [1.1000, 1.2000]])
Head Embeddings:
 tensor([[[0.1000, 0.2000],
         [0.1000, 0.2000],
         [0.1000, 0.2000]],

        [[0.3000, 0.4000],
         [0.5000, 0.6000],
         [0.1000, 0.2000]]])
Tail Embeddings:
 tensor([[[0.7000, 0.8000],
         [0.9000, 1.0000],
         [1.1000, 1.2000]],

        [[0.9000, 1.0000],
         [0.9000, 1.0000],
         [0.9000, 1.0000]]])
size head Embeddings:
 torch.Size([2, 3, 2])
size Tail Embeddings:
 torch.Size([2, 3, 2])


In [ ]:
data = dataset[0]
plot_ratings_vs_time(data)
#print (data)
#print (data['user', 'rates', 'movie'].rating[:20])
#print (graph["movie"].x[:10,:])


In [9]:
import torch
t_ranking = torch.zeros(5)
h_ranking = torch.zeros(6)
rankings1 = [t_ranking, h_ranking]
print(rankings1)
rankings2 = torch.cat([t_ranking, h_ranking], dim=0)
print(rankings2)


[tensor([0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0.])]
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [8]:

user_features = graph["user"].x
print (movies_features[:10])

tensor([[0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])


In [ ]:
import os
import torch

# Define your input and output directories.
input_dir = "/itet-stor/trachsele/net_scratch/tl4rec/ckpts/pretrain_full_ckpt"
output_dir = "/itet-stor/trachsele/net_scratch/tl4rec/ckpts/pretrain"
os.makedirs(output_dir, exist_ok=True)

# Loop over each checkpoint file in the input directory.
for ckpt_file in os.listdir(input_dir):
    # Only process files ending with .pth (or other extensions you expect)
    if not ckpt_file.endswith(".pth"):
        continue

    input_path = os.path.join(input_dir, ckpt_file)
    print(f"Processing {input_path} ...")
    
    # Load the full checkpoint.
    checkpoint = torch.load(input_path, map_location="cpu")
    full_state_dict = checkpoint["model"]

    # Extract the backbone (ultra) state_dict.
    backbone_state_dict = {}
    for key, value in full_state_dict.items():
        if key.startswith("ultra."):
            # Remove the "ultra." prefix.
            new_key = key[len("ultra."):]
            backbone_state_dict[new_key] = value

    # Construct the new checkpoint.
    new_checkpoint = {
        "model": backbone_state_dict,
        "optimizer": checkpoint["optimizer"]
    }
    
    output_path = os.path.join(output_dir, ckpt_file)
    torch.save(new_checkpoint, output_path)
    print(f"Saved backbone checkpoint to {output_path}")


In [3]:
import sqlite3
import pandas as pd


DB_FILE = "//itet-stor/trachsele/net_scratch/tl4rec/model_outputs/results_test.db" 
# Connect to the database
conn = sqlite3.connect(DB_FILE)

# Read all entries
df = pd.read_sql("SELECT * FROM experiments", conn)

# Close the connection
conn.close()

# Display the results

print(df)  # Use this if running in a terminal


                   ckpt dataset  epochs  bpe  FT          valid_mr  \
0  inionsBeautyMl1m.pth  LastFM       0  381   0  75.5225448608398   
1  inionsBeautyMl1m.pth  LastFM       1  381   0  72.0636520385742   

           valid_mrr        valid_hits@1       valid_hits@3  \
0  0.152451619505882  0.0768725946545601  0.152057752013206   
1  0.206827893853188   0.123089902102947   0.21664947271347   

       valid_hits@10  valid_ndcg@10           test_mr            test_mrr  \
0  0.297740697860718       0.293851  341.540466308594  0.0518215708434582   
1  0.374800771474838       0.381007  375.539276123047  0.0514721013605595   

          test_hits@1         test_hits@3       test_hits@10  test_ndcg@10  
0  0.0175452511757612  0.0441409945487976  0.119561761617661      0.342506  
1  0.0180215928703547  0.0457288026809692  0.117259442806244      0.345600  


In [ ]:
%run script/pretrain.py -c config/recommender/pretrain_all.yaml --gpus [0]

In [1]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset LastFM --epochs 1 --bpe 0 --gpus "[0]" --ckpt null

/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
16:47:59   Random seed: 1024
16:47:59   Config file: config/recommender/notebook_cfg.yaml
16:47:59   {'checkpoint': None,
 'dataset': {'class': 'LastFM',
             'root': '/itet-stor/trachsele/net_scratch/tl4rec/model_outputs/data'},
 'model': {'backbone_model': {'embedding_edge': {'dropout_rate': 0.03247182903513618,
                                                 'hidden_dims': [16,
                                                                 16,
                                                                 16,
                                                                 16,
                                                                 16,
                                            

We will evaluate vs 1000 negatives
discarded node_features
bpe = 381
edge_attr.shape = torch.Size([79442, 1])


16:48:00   ------------------------------
16:48:00   Number of parameters: 86801
16:48:00   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:48:00   Epoch 0 begin


freeze my backbone
unfreeze my backbone
Load rspmm extension. This may take a while...


16:48:15   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:48:15   binary cross entropy: 1.58981
16:48:18   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:48:18   binary cross entropy: 0.630787
16:48:20   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:48:20   binary cross entropy: 0.663853
16:48:23   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:48:23   binary cross entropy: 0.603572
16:48:25   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:48:25   Epoch 0 end
16:48:25   ------------------------------
16:48:25   average loss: 0.660632
16:48:26   Save checkpoint to model_epoch_1.pth
16:48:26   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:48:26   Evaluate on valid
16:49:21   mr: 9.3164
16:49:21   mrr: 0.432505
16:49:21   hits@1: 0.286585
16:49:21   hits@3: 0.492922
16:49:21   hits@10: 0.745664
16:49:21   ndcg@10: 0.497183
16:49:21   Load checkpoint from model_epoch_1.pth
16:49:21   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:49:21   Evaluate on valid
16:50:26   mr: 86.5833
16:50:26   mrr: 0.177916
16:50:26   hits@1: 0.101997
16:50:26   hits@3: 0.186182
16:50:26   hits@10:

In [ ]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset Gowalla --epochs 0 --bpe 20000 --gpus "[0]" --ckpt null

In [ ]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset Epinions --epochs 1 --bpe 10 --gpus "[0]" --ckpt null

In [ ]:
import torch
from rspmm import generalized_rspmm

# Test input data
edge_index = torch.tensor([[0, 1, 2], [1, 2, 3]], dtype=torch.long)
edge_type = torch.tensor([0, 1, 2], dtype=torch.long)
edge_weight = torch.tensor([1.0, 1.0, 1.0], dtype=torch.float32)
edge_attr = torch.tensor([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]], dtype=torch.float32)  # Edge features
relation = torch.rand(3, 4)  # Random relation features
input_features = torch.rand(4, 4)  # Node input features

# Call the function
output = generalized_rspmm(edge_index, edge_type, edge_weight, edge_attr, relation, input_features)
print("Output:", output)


In [2]:
print(os.getcwd())
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 
print(os.getcwd())

NameError: name 'os' is not defined

In [7]:
import pickle


def inspect_cxtdict(cxtdict):
    print("Type:", type(cxtdict))
    print("Number of entries:", len(cxtdict))
    if isinstance(cxtdict, dict):
        print("Number of entries:", len(cxtdict))
        # Print a sample of the keys and values
        for i, key in enumerate(cxtdict.keys()):
            print(f"Key {i}: {key}, Value: {cxtdict[key]}")
            if i >= 9:  # Limit to first 10 entries
                break


def load_data(filename):
    try:
        with open(filename, "rb") as f:
            x= pickle.load(f)
    except:
        x = []
    return x
    
CXTDict = load_data('/itet-stor/trachsele/net_scratch/tl4rec/CARCA_Code_and_Data/Data/CXTDictSasRec_Games.dat')
inspect_cxtdict(CXTDict)
#print(CXTDict)
#print(CXTDict[(1,49583)])



Type: <class 'dict'>
Number of entries: 287107
Number of entries: 287107
Key 0: (352, 60), Value: [0.0, 0.5, 0.7419354838709677, 0.2857142857142857, 0.4767123287671233, 6.25]
Key 1: (1814, 647), Value: [0.0, 0.8333333333333334, 0.45161290322580644, 0.42857142857142855, 0.7863013698630137, 10.25]
Key 2: (1796, 619), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 3: (1504, 173), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 4: (1504, 364), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 5: (1504, 592), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 6: (1504, 785), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 7: (1504, 797), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.86027397

In [1]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: pitri (pitri-eth-z-rich). Use `wandb login --relogin` to force relogin


acc,▂▁▇▆▆▇█▇
loss,█▃▃▃▄▃▃▁
acc,0.83106
loss,0.0692
